In [4]:
import os
import re
from googletrans import Translator
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set retry delay and maximum retries
RETRY_DELAY = 2  # Time to wait before retrying
MAX_RETRIES = 3  # Maximum number of retries for translation failures

def translate_json_files(input_folder, output_folder, target_language):
    os.makedirs(output_folder, exist_ok=True)

    logging.info('Started translation process.')

    def translate_text(text, filename, retries=MAX_RETRIES, chunk_size=4000):
        """
        This function handles the translation of text with retries on failure.
        """
        translator = Translator()  # Initialize Translator for each file
        
        for attempt in range(retries):
            try:
                # Split the text into chunks for translation
                chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
                translated_chunks = [translator.translate(chunk, dest=target_language).text for chunk in chunks]
                return " ".join(translated_chunks)  # Return concatenated translated text
            except Exception as e:
                logging.error(f"Error translating '{filename}' on attempt {attempt + 1}/{retries}: {e}")
                if attempt < retries - 1:
                    logging.info(f"Retrying in {RETRY_DELAY} seconds...")
                    time.sleep(RETRY_DELAY)  # Wait before retrying
                else:
                    logging.error(f"Failed to translate '{filename}' after {retries} attempts.")
                    return False  # Return None if all retries fail

    def preserve_markdown_formatting(text, filename):
        """
        This function replaces parts of the markdown text with placeholders,
        translates the text, and then puts everything back together.
        """
        # Define regular expressions for Markdown elements to preserve
        patterns = {
            'latex_block': r'\\\[.*?\\\]',  # Skip LaTeX math block: \[ text \]
        }

        # Find all Markdown elements and replace them with placeholders
        placeholders = {}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text)
            for i, match in enumerate(matches):
                placeholder = f"__{key}_{i}__"
                placeholders[placeholder] = match
                text = text.replace(match, placeholder)

        # Translate the remaining plain text
        translated_text = translate_text(text, filename)

        # Replace the placeholders with the original Markdown elements
        if translated_text:
            for placeholder, original in placeholders.items():
                translated_text = translated_text.replace(placeholder, original)
        
        return translated_text

    def translate_file(filename):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        try:
            with open(input_path, 'r', encoding='utf-8') as f:
                data = f.read()
            
            logging.info(f"Translating {filename}")
            
            # Preserve Markdown formatting (including MathJax) and translate
            translated_data = preserve_markdown_formatting(data, filename)
            
            if translated_data:
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(translated_data)
                logging.info(f"{filename} translated and saved to {output_path}")
            else:
                logging.error(f"Failed to translate {filename}")
        except Exception as e:
            logging.error(f"Error reading or processing '{filename}': {e}")

    files_to_translate = [filename for filename in os.listdir(input_folder) if filename.endswith(".md")]

    # Set max_workers to a reasonable value (e.g., 10 or 20) to avoid overloading the system/API
    with ThreadPoolExecutor(max_workers=200) as executor:
        futures = []
        
        # Submit tasks for each file
        for filename in files_to_translate:
            future = executor.submit(translate_file, filename)
            futures.append(future)
        
        # Process completed futures as they finish
        for future in as_completed(futures):
            try:
                future.result()  # Retrieve result or catch exceptions
            except Exception as e:
                logging.error(f"An error occurred: {e}")

# Example usage
# input_folder = "/path/to/input/folder"
# output_folder = "/path/to/output/folder"
# target_language = "es"  # For Spanish
# translate_json_files(input_folder, output_folder, target_language)


In [5]:
import os

locales = ['ta', 'hi', 'es', 'de', 'ru', 'ja', 'fr', 'pt', 'zh-CN', 'ar', 'it', 'ko', 'te', 'kn']
locales = ['hi', 'es', 'de', 'ru', 'ja', 'fr', 'pt', 'zh-CN', 'ar', 'it', 'ko', 'te', 'kn']
base_folder = "/Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter"
input_base_folder = os.path.join(base_folder, 'en')

# Function to process files
def process_file(input_file_path, output_folder_path, target_language):
    # Add your code to handle translation here
    translate_json_files(input_file_path, output_folder_path, target_language)

# Function to recursively navigate through directories
def process_folders(input_folder, output_folder, locale):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            input_file_path = os.path.join(root, file)
            # Create corresponding output folder if not exists
            relative_path = os.path.relpath(input_file_path, input_folder)
            output_file_path = os.path.join(output_folder, relative_path)
            os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
            # Process each file
            process_file(input_file_path, output_file_path, locale)

for locale in locales:
    print(f'Started processing locale: {locale}')

    localefolder = 'zh' if locale == 'zh-CN' else locale
    # Process folders inside the 'en' directory
    for input_folder in os.listdir(input_base_folder):
        # Skip 'soldiers' folder (or any folder you wish to exclude)
        if input_folder.lower() in ['acceleration', 'angle']:
            print(f"Skipping folder: {input_folder}")
            continue

        # if input_folder.lower() in ['density']:
        input_folder_path = os.path.join(input_base_folder, input_folder)
            
            # Only process directories inside 'en'
        if os.path.isdir(input_folder_path):
            output_folder_path = os.path.join(base_folder, f'{localefolder}/{input_folder}')
            target_language = locale
            translate_json_files(input_folder_path, output_folder_path, target_language)
        # else:
        #     output_folder_path = os.path.join(base_folder, f'{localefolder}')
        #     target_language = locale
        #     translate_json_files(input_folder_path, output_folder_path, target_language)



    
    print(f'Finished processing locale: {locale}')


2025-03-22 09:43:32,013 - INFO - Started translation process.
2025-03-22 09:43:32,016 - INFO - Translating terawatt.md
2025-03-22 09:43:32,016 - INFO - Translating pferdestärke.md
2025-03-22 09:43:32,017 - INFO - Translating ton_of_refrigeration.md
2025-03-22 09:43:32,017 - INFO - Translating kilopond_meter_per_second.md
2025-03-22 09:43:32,017 - INFO - Translating watt.md
2025-03-22 09:43:32,018 - INFO - Translating planck_power.md
2025-03-22 09:43:32,018 - INFO - Translating erg_per_second.md
2025-03-22 09:43:32,018 - INFO - Translating power.md
2025-03-22 09:43:32,019 - INFO - Translating kilowatt.md
2025-03-22 09:43:32,019 - INFO - Translating btus_per_second.md
2025-03-22 09:43:32,019 - INFO - Translating megawatt.md
2025-03-22 09:43:32,019 - INFO - Translating gigawatt.md
2025-03-22 09:43:32,020 - INFO - Translating foot_pound_per_second.md
2025-03-22 09:43:32,020 - INFO - Translating horsepower.md
2025-03-22 09:43:32,020 - INFO - Translating joule_per_second.md
2025-03-22 09:43:

Started processing locale: hi


2025-03-22 09:43:33,022 - INFO - btus_per_second.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/power/btus_per_second.md
2025-03-22 09:43:33,464 - INFO - megawatt.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/power/megawatt.md
2025-03-22 09:43:33,855 - INFO - erg_per_second.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/power/erg_per_second.md
2025-03-22 09:43:33,958 - INFO - decibel_watt.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/power/decibel_watt.md
2025-03-22 09:43:34,122 - INFO - horsepower.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/power/horsepower.md
2025-03-22 09:43:34,597 - INFO - watt.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/power/watt.md
2025-03-22 09:43:35,125 - INFO - volt_ampere.md translated

Skipping folder: acceleration


2025-03-22 09:43:45,723 - INFO - slug.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/mass/slug.md
2025-03-22 09:43:45,723 - INFO - milligram.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/mass/milligram.md
2025-03-22 09:43:45,723 - INFO - carat.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/mass/carat.md
2025-03-22 09:43:45,752 - INFO - kilogram.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/mass/kilogram.md
2025-03-22 09:43:45,889 - INFO - grain.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/mass/grain.md
2025-03-22 09:43:46,054 - INFO - ounce.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/hi/mass/ounce.md
2025-03-22 09:43:46,054 - INFO - stone.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-

...

Skipping folder: acceleration


2025-03-22 10:15:40,370 - INFO - grain.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/mass/grain.md
2025-03-22 10:15:41,148 - INFO - tonne.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/mass/tonne.md
2025-03-22 10:15:41,148 - INFO - ounce.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/mass/ounce.md
2025-03-22 10:15:41,149 - INFO - kilogram.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/mass/kilogram.md
2025-03-22 10:15:41,236 - INFO - milligram.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/mass/milligram.md
2025-03-22 10:15:41,246 - INFO - microgram.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/mass/microgram.md
2025-03-22 10:15:41,246 - INFO - stone.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam

Skipping folder: angle


2025-03-22 10:22:54,203 - INFO - kilo_joule_per_second.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/energy/kilo_joule_per_second.md
2025-03-22 10:22:54,322 - INFO - newton_meter.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/energy/newton_meter.md
2025-03-22 10:22:54,863 - INFO - kilowatt_hour.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/energy/kilowatt_hour.md
2025-03-22 10:22:55,026 - INFO - therm.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/energy/therm.md
2025-03-22 10:22:55,211 - INFO - electronvolt.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/energy/electronvolt.md
2025-03-22 10:22:55,214 - INFO - BTU.md translated and saved to /Users/rakesh.c/rakesh/my/am/inayam/inayam-doc/unit-converter/kn/energy/BTU.md
2025-03-22 10:22:55,452 - INFO - quad.md trans

Finished processing locale: kn
